Latihan Membuat Model Klasifikasi Gambar
>melakukan klasifikasi kamar berantakan dan rapih

In [1]:
#check tensor flow version
import tensorflow as tf
print(tf.__version__)

2.16.1


In [2]:
# melakukan ekstraksi pada file zip
import zipfile,os
local_zip = 'dataset\messy-vs-clean-room.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()
 
base_dir = '/tmp/images'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

In [3]:
os.listdir('/tmp/images/train')

['clean', 'messy']

In [4]:
os.listdir('/tmp/images/val')

['clean', 'messy']

In [5]:
#menerapkan ImageDataGenerator untuk data latih dan data validasi.
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')
 
test_datagen = ImageDataGenerator(
                    rescale=1./255)

In [6]:
#siapkan data latih dan validasi dari kumpulan data gambar yang di-load dalam memori 
#melalui fungsi flow() berikut.

train_generator = train_datagen.flow_from_directory(
        train_dir,  # direktori data latih
        target_size=(150, 150),  # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4,
        # karena ini merupakan masalah klasifikasi 2 kelas, gunakan class_mode = 'binary'
        class_mode='binary')
 
validation_generator = test_datagen.flow_from_directory(
        validation_dir, # direktori data validasi
        target_size=(150, 150), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4, # karena ini merupakan masalah klasifikasi 2 kelas gunakan class_mode = 'binary'
        class_mode='binary')

Found 192 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [7]:
#membangun model Convolutional Neural Network (CNN)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

c:\Users\alifsfp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [8]:
#melihat summary dari arsitektur model yang telah kita buat.
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                     ┃ Output Shape             ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                  │ (None, 148, 148, 32)     │           896 │
├──────────────────────────────────┼──────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)     │ (None, 74, 74, 32)       │             0 │
├──────────────────────────────────┼──────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)                │ (None, 72, 72, 64)       │        18,496 │
├──────────────────────────────────┼──────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)   │ (None, 36, 36, 64)       │             0 │
├──────────────────────────────────┼──────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)                │ (None, 34, 34, 128)      │        73,856 │
├──────────────────────────────────┼──────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)   │ (None, 17, 17, 128)      │             0 │
├──────────────────────────────────┼──────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)                │ (None, 15, 15, 512)      │       590,336 │
├──────────────────────────────────┼──────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)   │ (None, 7, 7, 512)        │             0 │
├──────────────────────────────────┼──────────────────────────┼───────────────┤
│ flatten (Flatten)                │ (None, 25088)            │             0 │
├──────────────────────────────────┼──────────────────────────┼───────────────┤
│ dense (Dense)                    │ (None, 512)              │    12,845,568 │
├──────────────────────────────────┼──────────────────────────┼───────────────┤
│ dense_1 (Dense)                  │ (None, 1)                │           513 │
└──────────────────────────────────┴──────────────────────────┴───────────────┘

 Total params: 13,529,665 (51.61 MB)

 Trainable params: 13,529,665 (51.61 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# melakukan compile model tersebut menggunakan fungsi compile().
# compile model dengan 'adam' optimizer loss function 'binary_crossentropy' 
model.compile(loss='binary_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

In [10]:
#model fitting
#melatih model pada data masukan dan label yang bersesuaian

# latih model dengan model.fit 
model.fit(
      train_generator,
      steps_per_epoch=25,  # berapa batch yang akan dieksekusi pada setiap epoch
      epochs=20, # tambahkan epochs jika akurasi model belum optimal
      validation_data=validation_generator, # menampilkan akurasi pengujian data validasi
      validation_steps=5,  # berapa batch yang akan dieksekusi pada setiap epoch
      verbose=2)

Epoch 1/20


c:\Users\alifsfp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 - 28s - 1s/step - accuracy: 0.4600 - loss: 0.8579 - val_accuracy: 0.7000 - val_loss: 0.6929
Epoch 2/20


c:\Users\alifsfp\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


25/25 - 20s - 809ms/step - accuracy: 0.4457 - loss: 0.6649 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/20
25/25 - 16s - 650ms/step - accuracy: 0.4900 - loss: 0.6940 - val_accuracy: 0.5000 - val_loss: 0.6930
Epoch 4/20
25/25 - 15s - 605ms/step - accuracy: 0.4783 - loss: 0.6645 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/20
25/25 - 15s - 611ms/step - accuracy: 0.5100 - loss: 0.6940 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 6/20
25/25 - 14s - 572ms/step - accuracy: 0.5109 - loss: 0.6642 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/20
25/25 - 10s - 391ms/step - accuracy: 0.5600 - loss: 0.6926 - val_accuracy: 0.5000 - val_loss: 0.6930
Epoch 8/20
25/25 - 8s - 333ms/step - accuracy: 0.4565 - loss: 0.6648 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/20
25/25 - 13s - 508ms/step - accuracy: 0.4400 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6877
Epoch 10/20
25/25 - 10s - 396ms/step - accuracy: 0.5761 - loss: 0.6576 - val_accurac

In [13]:
#test model

import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tkinter as tk
from tkinter import filedialog

def browse_files():
    filename = filedialog.askopenfilename()
    return filename

def test_model(filename):
    if not filename:
        print("No file selected.")
        return
    
    # predicting images
    img = image.load_img(filename, target_size=(150,150))

    imgplot = plt.imshow(img)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])

    classes = model.predict(images, batch_size=10)  
    print("Selected file:", filename)
    if classes == 0:
        print('clean')
    else:
        print('messy')

# Load your model here
# model = ...

# Set up GUI
root = tk.Tk()
root.withdraw()  # Hide the root window

browse_button = tk.Button(root, text="Choose File", command=lambda: test_model(browse_files()))
browse_button.pack()

root.mainloop()
